In [198]:
# Installing the necessary libaries
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import re

In [199]:
# Loading the SIS_faculty_list csv
df = pd.read_csv("SIS_faculty_list.csv")

In [200]:
# First observations of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 19 columns):
 #   Column                                                                                                                         Non-Null Count  Dtype 
---  ------                                                                                                                         --------------  ----- 
 0   ID                                                                                                                             282 non-null    object
 1   Name                                                                                                                           284 non-null    object
 2   Location                                                                                                                       270 non-null    object
 3   Grade                                                                                                              

# **General Cleansing**

## Adjusting the Header Columns in CSV


In [201]:
# converting the column headers into lower case
df.columns = df.columns.str.lower()

# renaming column headers and changing to snake case
df = df.rename(columns = {'join\ndate':'join_date', 
                          'reports to':'reports_to',
                          'divison':'division',
                          'highest\nqualification\nlevel':'highest_qualification_level', 
                          'highest qualification':'qualification',
                          'courses taught- term 201510':'courses_taught_term_201510',
                          'document other professional certification critiera five years work experience teaching excellence professional certifications':'other_professional_certificate',
                          'major teaching field':'major_teaching_field'})

In [202]:
# Using For Loops to convert the values in the dataframe to upper case
loop = ['location','title', 'type', 'division', 'highest_qualification_level', 'major', 'university', 'criteria', 'major_teaching_field']
for value in loop:
    df[value] = df[value].str.upper()

In [203]:
# Converting the 'join_date' column into datetime
df["join_date"] = pd.to_datetime(df["join_date"])

In this code, we use the str.replace() method to replace all instances of 'Ph.D' (with or without a period) with 'Doctorate' in the 'Highest Qualification' column. The case=False argument is included to make the search case-insensitive, so we will match 'PhD', 'ph.d', 'Ph.d', and so on.

(the same except for the entire dataset).

In [204]:
# Using RegEx to remove all special characters and white spaces in the dataframe
regex = r'[^a-zA-Z0-9\s]+'
df = df.replace(to_replace=regex, value='', regex=True)

## **Dropping Data Columns** 

ML models containing irrelevent or reundant data poses risk of overfitting. In this instance, the 'lwd' column only contained 11 instances that were not null and cannot be a reliable source for machine learning modelling. 'all qualifications from profile' and 'qualification' were dropped in effort to simplify the dataset. 

In [205]:
# Removing 'LWD' due to ambiguity and limited data
# Removing 'all qualifications from profile' and 'qualification' for redundancy
df = df.drop(['lwd', 'all qualifications from profile', 'qualification'], axis=1)

## **Column-Specific Cleansing** 

In [206]:
## id
# Removing any values not containing 'LT', with the assumption the ID column is the primary key.
df = df[df['id'].str.contains('LT', na=False)]

In [207]:
## title
# removing 'BUSINESS' from the "title" column to avoid redundancy using RegEx.
df['title'] = df['title'].str.replace(r'\bBUSINESS\b', '', regex=True)

# Cleaning individual rows the above code did not pick up.
df['title'] = df['title'].replace({'FACULTYBUSINESS HR ADMIN': 'FACULTY HR ADMIN', 
                                             'FACULTY COMP LIT FACULTY': 'FACULTY COMP LIT',
            'FACULTYBUSINESS'         : 'FACULTY'})

In [208]:
## courses_taught_term_201510
# removing programme titles outside of course code using RegEx. 
regex = r"\b([A-Z]{3}\d{4})\w*\b"
df['courses_taught_term_201510'] = df['courses_taught_term_201510'].str.findall(regex)

In [209]:
## university
# manual cleanup
df['university'] = df['university'].replace({'EBUSINESS POLICY DEVELOPMENT' : 'E BUSINESS POLICY DEVELOPMENT', 
                                             'INTEGRTD TEXTILE APPAREL SCI' : 'INTEGRATED TEXTILE APPAREL SCI'})

In [210]:
## major
# manual cleanup
df['major'] = df['major'].replace({'FRIEDRICHSCHILLERUNIV JENA': 'FRIEDRICH SCHILLER UNI JENA', 
                                   'UNIVERSITYOF KERALA'       : 'UNIVERSITY OF KERALA',
                                   'FACULTYBUSINESS'           : 'FACULTY'})

In [211]:
## major_teaching_field
# manual cleanup
df['major_teaching_field'] = df['major_teaching_field'].replace({'MANAGEMENT HRMMARKETING'                  :'MANAGEMENT HRM MARKETING',
                                                                 'BUSINESS MANAGEMENT\nCORPORATE GOVERNANCE':'BUSINESS MANAGEMENT CORPORATE GOVERNANCE',
                                                                 'BUSINESS ADMINISTRATION\nMARKETING'       :'BUSINESS ADMINISTRATION\nMARKETING'})

In [212]:
## highest_qualification_level
# standardising highest_qualification_level by removing values outside of PHD, DOCTORATE, MASTERS, BACHELOR
clean_qualifications = ['PHD', 'MASTERS', 'DOCTORATE', 'BACHELOR']
df['highest_qualification_level'] = df['highest_qualification_level'].str.upper().where(df['highest_qualification_level'].isin(clean_qualifications))

## **Addressing NaN Values** 

Several solutions were considered to address the NaN values in the dataset, being:

1.   Mean, median, or mode.
2.   Dropping NaN rows.
3.   Creating a separate category as 'UNKNOWN' to preserve observations. 




In [213]:
df = df.fillna('UNKNOWN')

In [214]:
# changing datapoints with no 'length to 'NO COURSE CODE'
df['courses_taught_term_201510'] = df['courses_taught_term_201510'].fillna('NO COURSE CODE')

In [215]:
## other_professional_certificate
# using regex to pull key words and numbers while preserving information
key_words = r'\b\d+\s+(?:years|months)|\bteaching|\btaught|\bexperience|\bindustry|\bprofessional|\bUNKNOWN|\bmanagement|\bCMA|\bGTC|\bLicensed Agriculturist|\bHCIMA CTHCM|\btraining|\bCert IV Australia PPC License|\bsemester|\bbusiness|\bMore than|\bcorporate|\beducation|\bcertificate|\bmember|\bhospitality|\bcertified\b'

# defining a function to pull the relevant string information 
def extract_info(string):
    matches = re.findall(key_words, string, flags=re.IGNORECASE)
    return ' '.join(matches)

# applying the function to the 'other_professional_certificate' column
df['other_professional_certificate'] = df['other_professional_certificate'].apply(extract_info)

# NOTE: the initial 'other_professional_certificate' column had 22.4% values as NaN which may compromise the ML algorithm.
# Worth having a discussion with relevent stakeholder due to pre-existing small sample size over uncertainty on whether to keep or remove. 

In [216]:
## integer encoding
encoded_values = ['location', 'title', 'highest_qualification_level', 'criteria']

for col in encoded_values:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])
    mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
    print(f"Integer Encoding For {col}: {mapping}")

# One-hot encoding generates too many dimenions not suitable for ML purposes
# Considered integer encoding other columns, however, due to the unique values therefore will keep as string values
# Many solutions were considered or whether this step was necessary due to being able to categorise using decision trees or SVM, given the task is to preprocess the dataset suitable for ML

Integer Encoding For location: {'BELFAST': 0, 'BIRMINGHAM': 1, 'BOLTON': 2, 'BRIGHTON': 3, 'BRISTOL': 4, 'CAMBRIDGE': 5, 'CARDIFF': 6, 'GLASGOW': 7, 'LEEDS': 8, 'LEICESTER': 9, 'LIVERPOOL': 10, 'LONDON': 11, 'MANCHESTER': 12, 'OXFORD': 13, 'SOUTHAMPTON': 14, 'UNKNOWN': 15}
Integer Encoding For title: {' COMP  MATH FACULTY': 0, 'ADJUNCT FACULTY  ': 1, 'FACULTY': 2, 'FACULTY ': 3, 'FACULTY  ': 4, 'FACULTY   ': 5, 'FACULTY    COMP LIT': 6, 'FACULTY    COMP LIT FACULTY': 7, 'FACULTY   ADMIN': 8, 'FACULTY  TRAINEE': 9, 'FACULTYBUSINESS HR  ADMIN': 10, 'GRADUATE TRAINEE': 11}
Integer Encoding For highest_qualification_level: {'BACHELOR': 0, 'DOCTORATE': 1, 'MASTERS': 2, 'PHD': 3, 'UNKNOWN': 4}
Integer Encoding For criteria: {'CERTIFICATION': 0, 'INDUSTRY  TEACHING EXPERIENCE': 1, 'RESEARCHER': 2, 'TRAINEE': 3}


## **Reviewing the Dataset** 

In [217]:
# print(df.to_string())
print(df.to_string())

            id                    name  location grade  title  join_date       type  division        reports_to  highest_qualification_level                          major                                                         university                                       courses_taught_term_201510                                                                                   major_teaching_field                                                                                           other_professional_certificate  criteria
0    LT9210743              Jim Hunter        11    FA      4 2010-08-29  FULL TIME  BUSINESS       Abbas Houda                            2          INTERNATIONAL AFFAIRS                                      AMERICAN UNIVERSITY OF BEIRUT                             [BUS2403, BUS2003, LSG2013, BUS1003]                                                                                BUSINESS ADMINISTRATION                              10 Years Business 10 Years

In [218]:
df.to_csv('cleaned_SIS_faculty_list.csv', index=False)